### Import Required Libraries and Set Up Environment Variables

In [1]:
# Import libraries and Set up environment
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json


In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Access the API key environment variable
# Set the base URL

url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)
print(query_url)

https://api.nytimes.com/svc/search/v2/articlesearch.json?api-key=None&begin_date=20130101&end_date=20230531&fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&sort=newest&fl=headline,web_url,snippet,source,keywords,pub_date,byline,word_count


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0,20):
    
    # create query with a page number
    # API results show 10 articles at a time
    page_query_url = f"{query_url}&page={page}" 

    # Make a "GET" request and retrieve the JSON
    response = requests.get(page_query_url).json()
    
    # add a twelve second interval between queries to avoid hitting the API too quickly
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    try:
        reviews = response
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        print(f"Checked page {page}") 

        # loop through the reviews["response"]["docs"] and append each review to the list

        
        # Print the page that was just retrieved
    except:     
        print(f"Page {page} not retrieved")
        break
        # Print the page number that had no results then break from the loop
    print(f"Total reviews retrieved: {len(reviews_list)}")
         


Page 0 not retrieved


In [5]:
# Preview the first 5 results in JSON format
num_reviews_to_preview = min(5, len(reviews_list))
for review in reviews_list[:num_reviews_to_preview]:
    print(json.dumps(review, indent=4))
# Use json.dumps with argument indent=4 to format data

print(json.dumps(reviews_list, indent=4))


[]


In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

""


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

reviews_df.head()

KeyError: 'headline.main'

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries’
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,NaN
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever’
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love’
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children’


In [ ]:
reviews_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization', 'title'],
      dtype='object')

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
for title in reviews_df["title"].to_list():
    print(title)

The Attachment Diaries’
nan
You Can Live Forever’
A Tourist’s Guide to Love’
Other People’s Children’
One True Loves’
The Lost Weekend: A Love Story’
A Thousand and One’
Your Place or Mine’
Love in the Time of Fentanyl’


### Access The Movie Database API

In [ ]:
### Access The Movie Database API
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to manage the rate limit
request_counter = 0

# Loop through the titles
for title in movie_list:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        time.sleep(10)
    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    response = requests.get(url, params={'api_key': tmdb_api_key, 'query': title})

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    if response.status_code != 200:
        print(f"Error fetching data for {title}: {response.status_code} - {response.text}")
        continue

    try:
        movie_data = response.json()
        if movie_data['results']:
            # Get movie ID
            movie_id = movie_data['results'][0]['id']
            # Make a request for the movie details
            movie_detail_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            details_response = requests.get(movie_detail_url)
            if details_response.status_code == 200:
                # Extract the details
                movie_details = details_response.json()

                # Extract genre names, spoken languages, and production countries
                genre_names = [genre['name'] for genre in movie_details.get('genres', [])]
                spoken_languages = [language['name'] for language in movie_details.get('spoken_languages', [])]
                production_countries = [country['name'] for country in movie_details.get('production_countries', [])]

                # Prepare the movie info dictionary
                movie_info = {
                    'title': title,
                    'genres': genre_names,
                    'languages': spoken_languages,
                    'countries': production_countries,  
                    'release_date': movie_details['release_date'],
                    'runtime': movie_details['runtime'],
                    'vote_average': movie_details['vote_average'],
                    'budget': movie_details['budget'],
                    'revenue': movie_details['revenue'],
                    'overview': movie_details['overview'],
                    'tagline': movie_details['tagline'],
                    'imdb_id': movie_details['imdb_id'],
                    'poster_path': movie_details['poster_path'],
                    'backdrop_path': movie_details['backdrop_path'],
                    'original_language': movie_details['original_language'],
                    'original_title': movie_details['original_title'],
                    'popularity': movie_details['popularity'],
                    'vote_count': movie_details['vote_count'],
                    'homepage': movie_details['homepage'],
                    'spoken_languages': movie_details['spoken_languages'],
                    'production_countries': movie_details['production_countries'],
                        
                }       

                # Append the movie info to the list
                tmdb_movies_list.append(movie_info)

                print(f"Found {title}")
            else:
                print(f"Details not found for {title}")
        else:
            print(f"No results for {title}")
    except Exception as e:
        print(f"Exception for {title}: {e}")

# Print the tmdb_movies_list
print(tmdb_movies_list)



Found The Attachment Diaries’
Found nan
Found You Can Live Forever’
Found A Tourist’s Guide to Love’
Found Other People’s Children’
Found One True Loves’
Found The Lost Weekend: A Love Story’
Found A Thousand and One’
Found Your Place or Mine’
Found Love in the Time of Fentanyl’
[{'title': 'The Attachment Diaries’', 'genres': ['Drama', 'Mystery', 'Thriller', 'Horror'], 'languages': ['Español'], 'countries': ['Argentina'], 'release_date': '2021-10-07', 'runtime': 102, 'vote_average': 4.0, 'budget': 0, 'revenue': 0, 'overview': 'Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.', 'tagline': '', 'imdb_id': 'tt11359158', 'poster_path': '/hKZHk6I1p58ZeXbwjQo

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
import json
num_movies_to_preview = min(5, len(tmdb_movies_list))
for movie in tmdb_movies_list[:num_movies_to_preview]:
    print(json.dumps(movie, indent=4))
    print()

{
    "title": "The Attachment Diaries\u2019",
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "languages": [
        "Espa\u00f1ol"
    ],
    "countries": [
        "Argentina"
    ],
    "release_date": "2021-10-07",
    "runtime": 102,
    "vote_average": 4.0,
    "budget": 0,
    "revenue": 0,
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "tagline": "",
    "imdb_id": "tt11359158",
    "poster_path": "/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg",
    "backdrop_path": null,
    "original_language": "es",
    "original_title": "El apego",
    "popularity": 1.101,
    "vote_count": 2,


In [ ]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list) 
tmdb_movies_df.head()


,title,genres,languages,countries,release_date,runtime,vote_average,budget,revenue,overview,tagline,imdb_id,poster_path,backdrop_path,original_language,original_title,popularity,vote_count,prodction_companies
0,The Attachment Diaries’,"[Drama, Mystery, Thriller, Horror]",[Español],[Argentina],2021-10-07,102,4.000,0,0,"Argentina, 1970s. A desperate young woman goes...",,tt11359158,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,None,es,El apego,1.101,2,[]
1,NaN,"[Drama, Crime]","[Nederlands, ]","[Belgium, Iraq]",2008-09-08,30,5.300,10000,0,Sahim Omar Kalifa's graduating Film. A Kurdish...,,tt3790100,/94sFc0dpiALW4kTEQ6rzz5pezMO.jpg,/lKV7zxryvsEn73O588eVSqn2q4s.jpg,ku,Nan,0.600,3,"[{'id': 125930, 'logo_path': None, 'name': 'Ho..."
2,You Can Live Forever’,"[Drama, Romance]","[English, Français]",[Canada],2023-03-24,96,6.750,0,15055,"When Jaime, a gay teenager, is sent to live in...",,tt15782570,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,en,You Can Live Forever,6.793,28,"[{'id': 63971, 'logo_path': None, 'name': 'Pro..."
3,A Tourist’s Guide to Love’,"[Romance, Comedy]","[English, Tiếng Việt]",[United States of America],2023-04-21,96,6.365,0,0,"After an unexpected break up, a travel executi...",,tt20115096,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,en,A Tourist's Guide to Love,22.076,144,"[{'id': 48782, 'logo_path': None, 'name': 'Hea..."
4,Other People’s Children’,"[Drama, Comedy]","[English, Français]",[France],2022-09-21,104,6.900,0,84178,"Rachel loves her life, her students, her frien...",,tt15403712,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,/33UGFiijBDX18NHecj5xz5GkkHt.jpg,fr,Les Enfants des autres,17.324,169,"[{'id': 7117, 'logo_path': '/mVAjfnm0GI1V6ZHiG..."


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
# Use an inner join to keep only the movies that are in both DataFrames
import pandas as pd
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title", how="inner")   
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,tagline,imdb_id,poster_path,backdrop_path,original_language,original_title,popularity,vote_count,prodction_companies
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject Movies;creative_works The Attachment D...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",,tt11359158,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,None,es,El apego,1.101,2,[]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject Movies;persons Kapur, Shekhar;persons ...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,Sahim Omar Kalifa's graduating Film. A Kurdish...,,tt3790100,/94sFc0dpiALW4kTEQ6rzz5pezMO.jpg,/lKV7zxryvsEn73O588eVSqn2q4s.jpg,ku,Nan,0.600,3,"[{'id': 125930, 'logo_path': None, 'name': 'Ho..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject Movies;creative_works You Can Live For...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",,tt15782570,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,en,You Can Live Forever,6.793,28,"[{'id': 63971, 'logo_path': None, 'name': 'Pro..."
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject Movies;creative_works A Tourists Gue t...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",,tt20115096,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,en,A Tourist's Guide to Love,22.076,144,"[{'id': 48782, 'logo_path': None, 'name': 'Hea..."
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject Movies;persons Zlotowski, Rebecca;crea...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",,tt15403712,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,/33UGFiijBDX18NHecj5xz5GkkHt.jpg,fr,Les Enfants des autres,17.324,169,"[{'id': 7117, 'logo_path': '/mVAjfnm0GI1V6ZHiG..."


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genres", "languages", "countries", "production_companies", "production_countries", "spoken_languages", "keywords", "cast", "crew",   "belongs_to_collection", "production_companies", "production_countries", "spoken_languages", "keywords", "cast", "crew", "belongs_to_collection"]

# Create a list of characters to remove
characters_to_remove = ["[", "]", "'", '"', "{", "}", "name", ":", "id","'"]
                        

# Loop through the list of columns to fix
for column in columns_to_fix:
    if column in tmdb_movies_df.columns:
    # Loop through the list of characters to remove for tmdb_movies_df
        for character in characters_to_remove:
        # Remove the character from tmdb_movies_df
            tmdb_movies_df[column] = tmdb_movies_df[column].astype(str).str.replace(character, "")

    # Loop through the list of characters to remove for reviews_df
    if column in reviews_df.columns:
        for character in characters_to_remove:
        # Remove the character from reviews_df
            reviews_df[column] = reviews_df[column].astype(str).str.replace(character, "")

# Display the fixed DataFrame
tmdb_movies_df.head()


,title,genres,languages,countries,release_date,runtime,vote_average,budget,revenue,overview,tagline,imdb_id,poster_path,backdrop_path,original_language,original_title,popularity,vote_count,prodction_companies
0,The Attachment Diaries’,"Drama, Mystery, Thriller, Horror",Español,Argentina,2021-10-07,102,4.000,0,0,"Argentina, 1970s. A desperate young woman goes...",,tt11359158,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,None,es,El apego,1.101,2,[]
1,NaN,"Drama, Crime","Nederlands,","Belgium, Iraq",2008-09-08,30,5.300,10000,0,Sahim Omar Kalifa's graduating Film. A Kurdish...,,tt3790100,/94sFc0dpiALW4kTEQ6rzz5pezMO.jpg,/lKV7zxryvsEn73O588eVSqn2q4s.jpg,ku,Nan,0.600,3,"[{'id': 125930, 'logo_path': None, 'name': 'Ho..."
2,You Can Live Forever’,"Drama, Romance","English, Français",Canada,2023-03-24,96,6.750,0,15055,"When Jaime, a gay teenager, is sent to live in...",,tt15782570,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,en,You Can Live Forever,6.793,28,"[{'id': 63971, 'logo_path': None, 'name': 'Pro..."
3,A Tourist’s Guide to Love’,"Romance, Comedy","English, Tiếng Việt",United States of America,2023-04-21,96,6.365,0,0,"After an unexpected break up, a travel executi...",,tt20115096,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,en,A Tourist's Guide to Love,22.076,144,"[{'id': 48782, 'logo_path': None, 'name': 'Hea..."
4,Other People’s Children’,"Drama, Comedy","English, Français",France,2022-09-21,104,6.900,0,84178,"Rachel loves her life, her students, her frien...",,tt15403712,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,/33UGFiijBDX18NHecj5xz5GkkHt.jpg,fr,Les Enfants des autres,17.324,169,"[{'id': 7117, 'logo_path': '/mVAjfnm0GI1V6ZHiG..."


In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,tagline,imdb_id,poster_path,backdrop_path,original_language,original_title,popularity,vote_count,prodction_companies
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject Movies;creative_works The Attachment D...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",,tt11359158,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,None,es,El apego,1.101,2,[]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject Movies;persons Kapur, Shekhar;persons ...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,Sahim Omar Kalifa's graduating Film. A Kurdish...,,tt3790100,/94sFc0dpiALW4kTEQ6rzz5pezMO.jpg,/lKV7zxryvsEn73O588eVSqn2q4s.jpg,ku,Nan,0.600,3,"[{'id': 125930, 'logo_path': None, 'name': 'Ho..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject Movies;creative_works You Can Live For...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",,tt15782570,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,en,You Can Live Forever,6.793,28,"[{'id': 63971, 'logo_path': None, 'name': 'Pro..."
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject Movies;creative_works A Tourists Gue t...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",,tt20115096,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,en,A Tourist's Guide to Love,22.076,144,"[{'id': 48782, 'logo_path': None, 'name': 'Hea..."
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject Movies;persons Zlotowski, Rebecca;crea...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",,tt15403712,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,/33UGFiijBDX18NHecj5xz5GkkHt.jpg,fr,Les Enfants des autres,17.324,169,"[{'id': 7117, 'logo_path': '/mVAjfnm0GI1V6ZHiG..."


In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates(subset=["title"])
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,tagline,imdb_id,poster_path,backdrop_path,original_language,original_title,popularity,vote_count,prodction_companies
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject Movies;creative_works The Attachment D...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",,tt11359158,/hKZHk6I1p58ZeXbwjQok7DSWfZ.jpg,None,es,El apego,1.101,2,[]
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject Movies;persons Kapur, Shekhar;persons ...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,...,Sahim Omar Kalifa's graduating Film. A Kurdish...,,tt3790100,/94sFc0dpiALW4kTEQ6rzz5pezMO.jpg,/lKV7zxryvsEn73O588eVSqn2q4s.jpg,ku,Nan,0.600,3,"[{'id': 125930, 'logo_path': None, 'name': 'Ho..."
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject Movies;creative_works You Can Live For...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",,tt15782570,/10MJURrsPfDQmS2AP3fueOHJ0ch.jpg,/sClvKM8C6a1DSPzMwkKjk8pNFDx.jpg,en,You Can Live Forever,6.793,28,"[{'id': 63971, 'logo_path': None, 'name': 'Pro..."
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject Movies;creative_works A Tourists Gue t...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,...,"After an unexpected break up, a travel executi...",,tt20115096,/uWkpjbBe4gRZilXRXbYfsMUZMhz.jpg,/z3Qc5a0grcCREAQKqI195P05dVf.jpg,en,A Tourist's Guide to Love,22.076,144,"[{'id': 48782, 'logo_path': None, 'name': 'Hea..."
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject Movies;persons Zlotowski, Rebecca;crea...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"Rachel loves her life, her students, her frien...",,tt15403712,/4vsQ4894pzd7nbvP5FoSOHaS1mI.jpg,/33UGFiijBDX18NHecj5xz5GkkHt.jpg,fr,Les Enfants des autres,17.324,169,"[{'id': 7117, 'logo_path': '/mVAjfnm0GI1V6ZHiG..."


In [ ]:
# Export data to CSV without the index
merged_df.to_csv('merged_df.csv', index=False)
print("Exported merged_df.csv")
           


Exported merged_df.csv
